In [1]:
# Core
import numpy as np
import pandas as pd
from scipy.interpolate import CubicSpline

In [2]:
# Local imports
import kepler_sieve
# from asteroid_element import get_asteroids
from asteroid_data import load_ast_vectors, load_ast_data, ast_add_earth_pos, ast_add_sun_vectors
from asteroid_spline import spline_ast_vec, spline_ast_elt, test_ast_spline_elt
from orbital_element_test import get_test_elements, report_test
from db_config import db_engine
from db_utils import sp2df
import matplotlib.pyplot as plt

In [3]:
df_ast = load_ast_data(n0=1, n1=11)

In [4]:
# Number of asteroids
N_ast = np.unique(df_ast.AsteroidID.values).size
# Number of times per asteroid
N_t = df_ast.shape[0] // N_ast

In [5]:
df_ast

,TimeID,AsteroidID,mjd,qx,qy,qz,vx,vy,vz,a,e,inc,Omega,omega,f,M
0,69120000,1,48000.0,-1.155142,2.297488,0.282872,-0.009457,-0.005464,0.001583,2.768788,0.077540,0.185062,1.408422,1.237946,-0.601468,-0.517779
1,69125760,1,48004.0,-1.192813,2.275319,0.289164,-0.009377,-0.005620,0.001563,2.768752,0.077525,0.185061,1.408417,1.237928,-0.584374,-0.502840
2,69131520,1,48008.0,-1.230156,2.252526,0.295376,-0.009294,-0.005776,0.001543,2.768714,0.077510,0.185060,1.408411,1.237912,-0.567257,-0.487900
3,69137280,1,48012.0,-1.267161,2.229116,0.301507,-0.009208,-0.005929,0.001522,2.768677,0.077495,0.185059,1.408406,1.237895,-0.550118,-0.472959
4,69143040,1,48016.0,-1.303818,2.205093,0.307555,-0.009120,-0.006082,0.001501,2.768639,0.077479,0.185057,1.408400,1.237879,-0.532956,-0.458018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37505,90696960,10,62984.0,0.728575,3.372451,0.098143,-0.008660,0.001518,-0.000541,3.147844,0.104631,0.066710,-1.344509,-0.805467,3.506618,-2.696021
37506,90702720,10,62988.0,0.693892,3.378329,0.095975,-0.008681,0.001421,-0.000543,3.147839,0.104631,0.066711,-1.344511,-0.805428,3.516782,-2.683744
37507,90708480,10,62992.0,0.659129,3.383816,0.093796,-0.008700,0.001323,-0.000546,3.147833,0.104631,0.066711,-1.344513,-0.805389,3.526955,-2.671469
37508,90714240,10,62996.0,0.624289,3.388913,0.091606,-0.008719,0.001225,-0.000549,3.147828,0.104632,0.066711,-1.344515,-0.805349,3.537136,-2.659194


In [6]:
interval = 24*60*8
mask = (df_ast.TimeID % interval == 0)

In [7]:
elt = df_ast[mask].copy()

In [9]:
elt[elt.AsteroidID==1]

,TimeID,AsteroidID,mjd,qx,qy,qz,vx,vy,vz,a,e,inc,Omega,omega,f,M
0,69120000,1,48000.0,-1.155142,2.297488,0.282872,-0.009457,-0.005464,0.001583,2.768788,0.077540,0.185062,1.408422,1.237946,-0.601468,-0.517779
2,69131520,1,48008.0,-1.230156,2.252526,0.295376,-0.009294,-0.005776,0.001543,2.768714,0.077510,0.185060,1.408411,1.237912,-0.567257,-0.487900
4,69143040,1,48016.0,-1.303818,2.205093,0.307555,-0.009120,-0.006082,0.001501,2.768639,0.077479,0.185057,1.408400,1.237879,-0.532956,-0.458018
6,69154560,1,48024.0,-1.376042,2.155230,0.319395,-0.008935,-0.006383,0.001458,2.768562,0.077449,0.185055,1.408389,1.237846,-0.498569,-0.428134
8,69166080,1,48032.0,-1.446743,2.102983,0.330881,-0.008739,-0.006678,0.001413,2.768484,0.077419,0.185053,1.408377,1.237814,-0.464100,-0.398247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3742,90673920,1,62968.0,0.555416,2.635466,-0.018423,-0.010355,0.001395,0.001950,2.767160,0.079043,0.184730,1.399220,1.259891,-1.297303,-1.147640
3744,90685440,1,62976.0,0.472315,2.645346,-0.002815,-0.010418,0.001074,0.001952,2.767149,0.079037,0.184730,1.399219,1.259914,-1.265771,-1.117776
3746,90696960,1,62984.0,0.388752,2.652644,0.012797,-0.010471,0.000750,0.001951,2.767136,0.079031,0.184730,1.399219,1.259938,-1.234092,-1.087912
3748,90708480,1,62992.0,0.304804,2.657332,0.028395,-0.010514,0.000422,0.001948,2.767123,0.079024,0.184731,1.399220,1.259960,-1.202266,-1.058048


In [8]:
# Choose a set of interpolation times
ts = np.arange(48000, 63001, 4.0)

In [12]:
from asteroid_spline import spline_ast_data
from orbital_element import unpack_elt_df, elt2vec, anomaly_M2f
from planets_interp import get_sun_vectors

In [28]:
# The columns to spline
cols_spline = ['a', 'e', 'inc', 'Omega', 'omega', 'Mx', 'My']

# Compute cosine and sine of M for splining
M_in = elt.M.values
elt['Mx'] = np.cos(M_in)
elt['My'] = np.sin(M_in)

# Spline these output columns
elt_out = spline_ast_data(df_ast=elt, ts=ts, cols_spline=cols_spline)
# Compute the splined M using atan2
elt_out['M'] = np.arctan2(elt_out.My, elt_out.Mx)
# Compute the splined f from M
elt_out['f'] = anomaly_M2f(M=elt.M, e=elt.e)

# Array of output times including repeated times for each asteroid
ts_out = elt_out.mjd.values

# Unpack orbital elements
a, e, inc, Omega, omega, f = unpack_elt_df(elt_out)

# Compute q and v in the heliocentric frame
q_hel, v_hel = elt2vec(a=a, e=e, inc=inc, Omega=Omega, omega=omega, f=f)

# Position and velocity of the sun
q_sun, v_sun = get_sun_vectors(ts=ts_out)

# The recovered position and velocity of the asteroid in the barycentric frame
q: np.array = q_hel + q_sun
v: np.array = v_hel + v_sun

# Build output state vectors; the two key columns in the output are AsteroidID and mjd
key_cols = elt_out.columns[0:2]
# Cartesian coordinate columns
cols_q = ['qx', 'qy', 'qz']
cols_v = ['vx', 'vy', 'vz']
# Need to copy from elts_out to avoid dreaded "setting with copy" warning in Pandas
vec_out = elt_out[key_cols].copy()
vec_out[cols_q] = q
vec_out[cols_v] = v

In [29]:
elt

,TimeID,AsteroidID,mjd,qx,qy,qz,vx,vy,vz,a,e,inc,Omega,omega,f,M,Mx,My
0,69120000,1,48000.0,-1.155142,2.297488,0.282872,-0.009457,-0.005464,0.001583,2.768788,0.077540,0.185062,1.408422,1.237946,-0.601468,-0.517779,0.868921,-0.494952
2,69131520,1,48008.0,-1.230156,2.252526,0.295376,-0.009294,-0.005776,0.001543,2.768714,0.077510,0.185060,1.408411,1.237912,-0.567257,-0.487900,0.883319,-0.468772
4,69143040,1,48016.0,-1.303818,2.205093,0.307555,-0.009120,-0.006082,0.001501,2.768639,0.077479,0.185057,1.408400,1.237879,-0.532956,-0.458018,0.896930,-0.442172
6,69154560,1,48024.0,-1.376042,2.155230,0.319395,-0.008935,-0.006383,0.001458,2.768562,0.077449,0.185055,1.408389,1.237846,-0.498569,-0.428134,0.909742,-0.415174
8,69166080,1,48032.0,-1.446743,2.102983,0.330881,-0.008739,-0.006678,0.001413,2.768484,0.077419,0.185053,1.408377,1.237814,-0.464100,-0.398247,0.921742,-0.387803
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37501,90673920,10,62968.0,0.866426,3.345069,0.106701,-0.008568,0.001904,-0.000529,3.147865,0.104630,0.066709,-1.344501,-0.805614,3.466036,-2.745134,-0.922434,-0.386154
37503,90685440,10,62976.0,0.797684,3.359532,0.102446,-0.008616,0.001712,-0.000535,3.147855,0.104630,0.066710,-1.344505,-0.805542,3.486312,-2.720576,-0.912674,-0.408689
37505,90696960,10,62984.0,0.728575,3.372451,0.098143,-0.008660,0.001518,-0.000541,3.147844,0.104631,0.066710,-1.344509,-0.805467,3.506618,-2.696021,-0.902364,-0.430974
37507,90708480,10,62992.0,0.659129,3.383816,0.093796,-0.008700,0.001323,-0.000546,3.147833,0.104631,0.066711,-1.344513,-0.805389,3.526955,-2.671469,-0.891512,-0.452997


In [23]:
np.arctan2(0.86, -0.49)

2.0886893088652876

In [30]:
elt_out[['Mx', 'My', 'M']]

,Mx,My,M
0,0.868921,-0.494952,-0.517779
1,0.876218,-0.481916,NaN
2,0.883319,-0.468772,-0.487900
3,0.890224,-0.455523,NaN
4,0.896930,-0.442172,-0.458018
...,...,...,...
37505,-0.902364,-0.430974,-2.696021
37506,-0.897006,-0.442019,NaN
37507,-0.891512,-0.452997,-2.671469
37508,-0.885885,-0.463906,NaN


In [19]:
elt_out.My

0       -0.494952
1       -0.481916
2       -0.468772
3       -0.455523
4       -0.442172
           ...   
37505   -0.430974
37506   -0.442019
37507   -0.452997
37508   -0.463906
37509   -0.474744
Name: My, Length: 37510, dtype: float64

In [7]:
# Spline orbital elements
df_elt = spline_ast_elt(elt=df_ast, ts=ts)

In [8]:
df_elt

,AsteroidID,mjd,qx,qy,qz,vx,vy,vz
0,1,48000.0,-1.155142,2.297488,0.282872,-0.009457,-0.005464,0.001583
1,1,48004.0,-1.192813,2.275319,0.289164,-0.009377,-0.005620,0.001563
2,1,48008.0,-1.230156,2.252526,0.295376,-0.009294,-0.005776,0.001543
3,1,48012.0,-1.267161,2.229116,0.301507,-0.009208,-0.005929,0.001522
4,1,48016.0,-1.303818,2.205093,0.307555,-0.009120,-0.006082,0.001501
...,...,...,...,...,...,...,...,...
37505,10,62984.0,0.728575,3.372451,0.098143,-0.008660,0.001518,-0.000541
37506,10,62988.0,0.693892,3.378329,0.095975,-0.008681,0.001421,-0.000543
37507,10,62992.0,0.659129,3.383816,0.093796,-0.008700,0.001323,-0.000546
37508,10,62996.0,0.624289,3.388913,0.091606,-0.008719,0.001225,-0.000549


In [9]:
# Spline vectors directly
df_vec = spline_ast_vec(vec=df_ast, ts=ts)

In [10]:
df_vec

,AsteroidID,mjd,qx,qy,qz,vx,vy,vz
0,1,48000.0,-1.155142,2.297488,0.282872,-0.009457,-0.005464,0.001583
1,1,48004.0,-1.192813,2.275319,0.289164,-0.009377,-0.005620,0.001563
2,1,48008.0,-1.230156,2.252526,0.295376,-0.009294,-0.005776,0.001543
3,1,48012.0,-1.267161,2.229116,0.301507,-0.009208,-0.005929,0.001522
4,1,48016.0,-1.303818,2.205093,0.307555,-0.009120,-0.006082,0.001501
...,...,...,...,...,...,...,...,...
37505,10,62984.0,0.728575,3.372451,0.098143,-0.008660,0.001518,-0.000541
37506,10,62988.0,0.693892,3.378329,0.095975,-0.008681,0.001421,-0.000543
37507,10,62992.0,0.659129,3.383816,0.093796,-0.008700,0.001323,-0.000546
37508,10,62996.0,0.624289,3.388913,0.091606,-0.008719,0.001225,-0.000549
